# Speech to Text Testing Notebook

Dependencies

Pip (env)
- google-cloud-speech[libcst]
- ffmpeg-python


Conda 
- nb_conda_kernels

Conda (env)
- Python 3.10
- ipykernel


## Imports

In [18]:
from google.cloud import speech
import ffmpeg
import sys
# from tqdm.notebook import tqdm

## Video Conversion

Using MP4 (lossy codec) can reduce the accuracy of the audio. In order to generate the best results, we will transcode the video into lossless codec (FLAC). I decided not to go with LINEAR16 since its use case lies in applications that need more higher quality audio (also results in higher file size)

## Optimal Formats
- Sampling Rate > 16k Hz\n
- Lossless Codec (FLAC)\n
- Short queries -> use `StreamingRecognize` with `single_utterance` set to `true`

In [19]:
import os
base_dir = os.getcwd()
print(base_dir)

C:\Users\ricky\Documents\My Files\Altum Labs\autolab\sst_transcription


In [20]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [21]:
# probe = ffmpeg.probe(base_dir+'/in.mp4')
# audio_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'audio']
# audio_streams #verifying that there is audio

In [26]:
!ffmpeg -i ../data/wetlab1/sec4.mp4 2>&1

ffmpeg version 2023-07-06-git-f00222e81f-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-liblensfun --en

In [29]:
# testing

input_dir = '../data/wetlab1/sec4.mp4'
output_dir = '../data/wetlab1_flac/output.flac'

# FLIPS VIDEO
# {
#     ffmpeg
#     .input('sec4.mp4')
#     .hflip()
#     .output('output1.mp4', acodec='copy') #acodec is supposed to copy the audio over but doesn't quite work --izzy
#     .run()
# }

#converting mp4 to audio flac file
{
    ffmpeg
    .input(input_dir)
    .output(output_dir, acodec='flac')
    .run()
}

{(None, None)}

In [27]:
!ffmpeg -i ../data/wetlab1_flac/output.flac 2>&1

ffmpeg version 2023-07-06-git-f00222e81f-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-liblensfun --en

## Speech2Text 

In [ ]:
client = speech.SpeechClient()

request = speech.RecognizeRequest(request={"config": config, "audio": audio})
response = client.list_voices(request=request)